In [1]:
# Read RData
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json
from os import listdir, getcwd
from os.path import isfile, join
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
import numpy as np

# Read all files in a directory
mypath = getcwd()+"/data/tweet_youtube"
onlyfiles = [ join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
tweets_time = {}
for f in onlyfiles:
    ro.r['load'](f,ro.globalenv)

shares = ro.globalenv[str('data')].rx2("numShare")
views = ro.globalenv[str('data')].rx2("dailyViewcount")

data = [ (shares[i][:120],views[i][:120])  for i in xrange(len(shares))]

In [2]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import learn
from sklearn.metrics import mean_squared_error
from lstm import generate_data, lstm_model, load_csvdata
import dateutil.parser
import datetime
import matplotlib.dates as mdates

In [3]:
LOG_DIR = './ops_logs/lstm_popularity'
TIMESTEPS = 6
RNN_LAYERS = [{'num_units': 20}]
DENSE_LAYERS = [10,TIMESTEPS]
TRAINING_STEPS = 4000
BATCH_SIZE = 220
PRINT_STEPS = TRAINING_STEPS / 10

In [ ]:
def loop_func(index):
    s_i = data[index][0]
    v_i = data[index][1]
    
    if(len(s_i)<120):
        return
        #         continue

    data_i = []
    for i in range(120):
        data_i.append([s_i[i]])
        data_i.append([v_i[i]])
    data_vs = pd.DataFrame(data_i, 
                           columns = ['data'])
    X, y = load_csvdata(data_vs, TIMESTEPS, seperate=False)
    
    regressor = learn.Estimator(model_fn=lstm_model(TIMESTEPS, RNN_LAYERS, DENSE_LAYERS,\
                                               optimizer = "Adagrad"))
    
    validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],
                                                         every_n_steps=PRINT_STEPS,
                                                         early_stopping_rounds=100)
    regressor.fit(X['train'], y['train'],
                  monitors=[validation_monitor],
                  batch_size=BATCH_SIZE,
                  steps=TRAINING_STEPS)

    predicted = 0
    pred = []
    test_x = X['test'][0]
    for ii in range(len(X['test'])):
        if(ii == 0):
            pass
        else:
            test_x = np.append(test_x, X['test'][ii][-1])
            test_x = np.reshape(test_x[2:],(-1,1))
        predicted = regressor.predict(np.array([test_x]))
        for p in predicted:
            pred.append(p)
            test_x = np.append(test_x,[p])
    pred_pop = sum(pred)+sum(v_i[:90])
    with open("results3.txt", "a") as myfile:
        myfile.write(str((index,pred_pop))+"\n")

from joblib import Parallel, delayed
pred = Parallel(n_jobs=20)(delayed(loop_func)(index) \
                                  for index in range(4083,10000))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe86de24810>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe86de1d690>, 'tf_config': gpu_options

In [ ]:
plot_predicted = plt.plot(pred, label='predicted')
plot_test = plt.plot(y['test'], label='test')
plt.legend(handles=plot_predicted+plot_test)

In [ ]:
print(s_i)

In [ ]:
def build_index2popularity_scale(index2shares):
    sorted_index2shares = sorted(index2shares, key=lambda tup: tup[1]) 
    split_index2shares = np.array_split(sorted_index2shares,40)
    
    # Build a hashmap : the index of a video -> popularity_scale
    index2ps = {}
    for scale in range(40):
        for element in split_index2shares[scale]:
            index2ps[element[0]] = scale
    
    return index2ps

total_size = len(shares)
sharecounts_120 = [(index, sum(shares[index][-120:])) for index in range(total_size)
                   if sum(shares[index][-120:])>=100] 
real_index2ps = build_index2popularity_scale(sharecounts_120)